Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*
    
# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [0]:

import pandas_profiling

#df.profile_report()

In [0]:
# Use a subset of the data where BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS' and the sale price was more than 100 thousand and less than 2 million.
df = df[df["BUILDING_CLASS_CATEGORY"] == "01 ONE FAMILY DWELLINGS"]
df = df[(100000 < df["SALE_PRICE"]) & (df["SALE_PRICE"] < 1000000)]

In [0]:
df = df.fillna(value=df.mean(skipna=True))

In [0]:
# Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
df["SALE_DATE"] = pd.to_datetime(df["SALE_DATE"])
train = df[(df["SALE_DATE"] <= "01/01/2019") & (df["SALE_DATE"] < "04/01/2019")]
test = df[(df["SALE_DATE"] <= "04/01/2019") & (df["SALE_DATE"] < "05/01/2019")]


In [0]:
# Do one-hot encoding of categorical features.
target = "SALE_PRICE"
high_cardinality = ["ADDRESS", "APARTMENT_NUMBER", "SALE_DATE", "EASE-MENT"]

X_train = train.drop(axis=1, labels=[target] + high_cardinality)
X_test = test.drop(axis=1, labels=[target] + high_cardinality)

y_train = train[target]
y_test = test[target]

import category_encoders as ce

encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [11]:
X_train.head()

,BOROUGH_3,BOROUGH_4,NEIGHBORHOOD_OTHER,BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,TAX_CLASS_AT_PRESENT_1,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT_A9,BUILDING_CLASS_AT_PRESENT_A1,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,"LAND_SQUARE_FEET_6,800","LAND_SQUARE_FEET_4,000",GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE_A9,BUILDING_CLASS_AT_TIME_OF_SALE_A1
44,1,0,1,1,1,5495,801,1,0,11230.0,1.0,0.0,1.0,1,0,1325.0,1930.0,1,1,0
61,0,1,1,1,1,7918,72,0,1,11427.0,1.0,0.0,1.0,0,1,2001.0,1940.0,1,0,1


In [12]:
# Do feature selection with SelectKBest.

from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

def getMAE(X_train, y_train, X_test, y_test, k=10):
    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    model = LinearRegression()
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)

    mae = mean_absolute_error(y_pred, y_test)
    return mae

maes = [getMAE(X_train, y_train, X_test, y_test, k=k) for k in range(1, len(X_train.columns)+1)]



/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: invalid value encountered in multiply
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: inva

In [13]:
# make a scree plot

from plotly import graph_objects as go

fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=list(range(1, len(maes)+1)),
        y=maes,
    )
)

fig.show()

In [14]:
# do it again with ridge selection
# Do feature selection with SelectKBest.

from sklearn.linear_model import RidgeCV, Ridge

def getMAE(X_train, y_train, X_test, y_test, k=10):
    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)


    model = RidgeCV(alphas=[10**(x/10.) for x in range(-40, 41)], normalize=True)
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)

    mae = mean_absolute_error(y_pred, y_test)
    return mae

maes = [getMAE(X_train, y_train, X_test, y_test, k=k) for k in range(1, len(X_train.columns)+1)]


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning:

divide by zero encountered in true_divide

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning:

invalid value encountered in multiply

/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning:

invalid value encountered in greater

/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning:

invalid value encountered in less

/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning:

invalid value encountered in less_equal

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning:

invalid value encounter

In [15]:
# make a scree plot

from plotly import graph_objects as go

fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=list(range(1, len(maes)+1)),
        y=maes,
    )
)

fig.show()

In [16]:
selector = SelectKBest(score_func=f_regression, k=7)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

model = RidgeCV(alphas=[10**x for x in range(-5, 6)], normalize=True)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_pred, y_test)

print(f"""
MAE: {mae}
alpha: {model.alpha_}
""")



MAE: 254033.94115550985
alpha: 0.01



/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning:

divide by zero encountered in true_divide

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning:

invalid value encountered in multiply

/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning:

invalid value encountered in greater

/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning:

invalid value encountered in less

/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning:

invalid value encountered in less_equal



In [18]:
alphas = [10.**(x/100.) for x in range(-600, -25)]
print("alphas")
print(alphas)

model = RidgeCV(alphas=alphas, normalize=True)
model.fit(X_train_selected, y_train)
y_pred = model.predict(X_test_selected)

mae = mean_absolute_error(y_pred, y_test)

print(f"""
MAE: {mae}
alpha: {model.alpha_}
""")

alphas
[1e-06, 1.0232929922807537e-06, 1.0471285480508985e-06, 1.071519305237607e-06, 1.096478196143185e-06, 1.122018454301963e-06, 1.1481536214968817e-06, 1.1748975549395304e-06, 1.2022644346174132e-06, 1.2302687708123811e-06, 1.2589254117941661e-06, 1.288249551693135e-06, 1.3182567385564074e-06, 1.3489628825916533e-06, 1.3803842646028838e-06, 1.4125375446227554e-06, 1.445439770745928e-06, 1.479108388168207e-06, 1.5135612484362072e-06, 1.5488166189124828e-06, 1.584893192461114e-06, 1.6218100973589298e-06, 1.6595869074375596e-06, 1.698243652461746e-06, 1.7378008287493763e-06, 1.778279410038923e-06, 1.8197008586099826e-06, 1.8620871366628656e-06, 1.9054607179632483e-06, 1.9498445997580454e-06, 1.9952623149688787e-06, 2.0417379446695273e-06, 2.0892961308540407e-06, 2.1379620895022326e-06, 2.1877616239495517e-06, 2.2387211385683376e-06, 2.2908676527677747e-06, 2.3442288153199226e-06, 2.3988329190194897e-06, 2.4547089156850284e-06, 2.5118864315095823e-06, 2.5703957827688647e-06, 2.63026799